# Horse Colic Dataset investigation

## Introduction to the Dataset

The dataset used for this investigation is the Horse Colic dataset from UC Irvine. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Business Questions

The following questions will be explored via the UC Irvine Horse Colic dataset. Each of these questions is relevant from an exploratory data analysis perspective when seeking to make predictions and decisions about a horse's prognosis and treatment in various colic cases. This topic is of interest and importance to horse owners, trainers, barn managers and veterinarians.

1. What characteristics are most associated with cases in which the colic needed surgery to be treated?
2. What characteristics of the horse's condition were most associated with a "lived" outcome (as opposed to "died" or "was euthanized")?
3. What were some of the ranges of the various health indicator parameters commonly measured by horse owners (such as rectal temperature, heart rate, respiratory rate, capillary refill time, mucous membranes and pain level)?


## Gather

The first step in the CRISP-DM Data Science process is "Gather", so we gather the dataset from UC Irvine by loading it into the notebook.

In [ ]:
# TODO: Load in dataset

# horse_data = pd.read_csv()

## Assess

The next step in the Data Science process is "Assess", so we examine the loaded data to identify categorical features, and missing values.

In [ ]:
# TODO: That

## Clean

The third step in this process is to "Clean", at which point we will handle categorical features and missing values.

In [ ]:
# TODO: That

## Analyze

Next, we will begin to answer the business questions outlined above, by analyzing the cleaned dataset and identifying the relevant features for answering each business question.

### Question 1: What characteristics are most associated with cases in which the colic needed surgery to be treated?

For this question, the relevant features to consider are:


In [ ]:
# TODO: That

### Question 2: What characteristics of the horse's condition were most associated with a "lived" outcome (as opposed to "died" or "was euthanized")?

For this question, we need to consider the following features:


In [ ]:
# TODO: That

### Question 3: What were some of the ranges of the various health indicator parameters commonly measured by horse owners (such as rectal temperature, heart rate, respiratory rate, capillary refill time, mucous membranes and pain level)?

For this question, the features we need to consider are:



In [ ]:
# TODO: That

## Model

For the "Model" step of the CRISP-DM process, we examine each business question individually and select the appropriate model for answering the question.

### Question 1: Surgery

To answer this question, we can perform a classification based on the features identified in the "Analyze" step and the target feature (whether or not the horse required surgery).

In [ ]:
# TODO: That

### Question 2: Life or Death

To answer this question, we can again perform a classification based on the features identified in the "Analyze" step and the target feature, this time considering two categories ("Died" and "Was Euthanized") as equivalent, since for the purposes of this question, they are.

In [ ]:
# TODO: That

### Question 3: Health Parameters

Many horse owners, trainers, barn managers and veterinarians check a particular subset of health parameters for horses, which can indicate colic or other health conditions. It is useful to know what the ranges for these health parameters were in the dataset for colic cases, since these can be compared to healthy ranges, which are pretty well established in the literature.

In [ ]:
# TODO: That

## Visualize

For the final step of this data science project, we will examine visualizations for each of the business questions.

### Question 1: Surgery

The following visualization of the model generated above shows the relationship between a variety of parameters and the resolution (surgical or medical) of the colic case.

In [ ]:
# TODO: That

## Question 2: Life or Death

The following is a visualization showing the relationship between various parameters and the outcome of the colic case ("Lived" or "Died"/"Was Euthanized").

In [ ]:
# TODO: That

## Question 3: Health Parameters

Finally, we include a visualization of the ranges of the various health parameters commonly checked by horse owners and others who come into regular contact with horses.

In [ ]:
# TODO: That